In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset-agent.csv")

# Display the first few rows of the DataFrame
print(df.head(1))

   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   

   agent_weight_a  agent_weight_b agent_trace_url  
0        0.317076        0.682924             NaN  


In [5]:
# force a fresh import of the module, otherwise you need to restart the ipython kernel whenever you make changes to the deepfunding.py file
import importlib
import deepfunding
importlib.reload(deepfunding)

from deepfunding import run_comparison
from tools.data_collection import fetch_repo_metrics

# Initialize agent weight columns if they don't exist
if 'agent_weight_a' not in df.columns:
    df['agent_weight_a'] = None
if 'agent_weight_b' not in df.columns:
    df['agent_weight_b'] = None
if 'agent_trace_url' not in df.columns:
    df['agent_trace_url'] = None

for i in range(len(df)):
    if pd.notnull(df.at[i, 'agent_weight_a']) and pd.notnull(df.at[i, 'agent_weight_b']):
        continue

    repo_a_url = df.iloc[i]["project_a"]
    repo_b_url = df.iloc[i]["project_b"]
    print(f"{repo_a_url} VS {repo_b_url}")
    repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }
    repo_b = { "url": repo_b_url, **fetch_repo_metrics(repo_b_url) }

    agent_result = run_comparison(repo_a, repo_b)

    df.at[i, 'agent_weight_a'] = agent_result['weights'][repo_a_url]
    df.at[i, 'agent_weight_b'] = agent_result['weights'][repo_b_url]
    df.at[i, 'agent_trace_url'] = agent_result['trace_url']

    print(f"Agent: {df.iloc[i]['agent_weight_a']} {df.iloc[i]['agent_weight_b']}")
    print(f"Human: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")
    print(f"Check details here: {df.iloc[i]['agent_trace_url']}")
    df.to_csv("mini-contest/dataset-agent.csv", index=False)

https://github.com/walletconnect/walletconnect-monorepo VS https://github.com/chzyer/readline
Project Analyzer Result: weight_a=0.85 weight_b=0.15 confidence=0.9 reasoning='Repo A has significantly higher activity metrics, including a large active developer count (30), a high number of commits (1954), and a substantial contributor count (2869). These metrics suggest strong ongoing development and a healthy community. In contrast, Repo B shows no recent activity or contributions, indicating a lack of community engagement and potential abandonment. Although Repo B has a decent star count (2105), it lacks the recent contributions and developer engagement that indicate sustainability. Therefore, I assign a higher weight to Repo A, reflecting its stronger metrics and potential for future growth and sustainability.' metrics_used=['activeDeveloperCount6Months', 'closedIssueCount6Months', 'commitCount6Months', 'contributorCount', 'forkCount', 'starCount', 'total_funding_received_usd']
Funding 

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT